<a href="https://colab.research.google.com/github/jstephencorey/bert-experiments/blob/main/bert_testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
on_colab = True
first_run = False # Change this when you disconnect and reconnect the runtime
if on_colab and first_run:
    !git clone https://github.com/jstephencorey/bert-experiments.git  bert_git # Only need to run this on colab
    from pathlib import Path
    import sys
    sys.path.append(str('/content/bert_git')) #Only need on the colab
    print(sys.path)
    !pip install transformers --quiet
    !pip install wandb --quiet

Cloning into 'bert_git'...
remote: Enumerating objects: 47, done.
remote: Counting objects: 100% (47/47), done.
remote: Compressing objects: 100% (45/45), done.
remote: Total 47 (delta 11), reused 16 (delta 1), pack-reused 0
Unpacking objects: 100% (47/47), done.
['/content', '/env/python', '/usr/lib/python38.zip', '/usr/lib/python3.8', '/usr/lib/python3.8/lib-dynload', '', '/usr/local/lib/python3.8/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.8/dist-packages/IPython/extensions', '/root/.ipython', '/content/bert_git']
     |████████████████████████████████| 5.8 MB 5.3 MB/s 
     |████████████████████████████████| 182 kB 40.0 MB/s 
     |████████████████████████████████| 7.6 MB 11.3 MB/s 
     |████████████████████████████████| 1.9 MB 4.4 MB/s 
     |████████████████████████████████| 174 kB 43.6 MB/s 
     |████████████████████████████████| 182 kB 38.8 MB/s 
     |████████████████████████████████| 62 kB 804 kB/s 
     |████████████████████████████████| 173 k

In [2]:
import torch
from torch import nn
import random
from torch import optim
import bert
import wandb
wandb.login()

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#Import the tokenizers

In [4]:
from transformers import AutoTokenizer

In [5]:
DATA_DIR = '/content/drive/MyDrive/AI_Data/'
MODEL_DIR = '/content/drive/MyDrive/AI_Models/storygpt/'
TOKENIZER_DIR = f'{MODEL_DIR}storygpt2tokenizer_ft2/'

bibliotik_train = f'{DATA_DIR}bibliotik_corpus/biblitik_22500_full.gz'
bibliotik_val = f'{DATA_DIR}bibliotik_corpus/biblitik_7500_val.gz'

atlas_shrugged_filename = f'{DATA_DIR}atlas-shrugged.txt'
anthem_filename = f'{DATA_DIR}anthem.txt'

In [6]:
BOS_TOKEN = '<BOS>'
EOS_TOKEN = '<EOS>'
PAD_TOKEN = '<PAD>'
CLS_TOKEN = '[CLS]'
MASK_TOKEN = '[MASK]'

tokenizer = AutoTokenizer.from_pretrained(TOKENIZER_DIR)

special_tokens_dict = {'cls_token':CLS_TOKEN, 'bos_token': BOS_TOKEN, 'eos_token': EOS_TOKEN, 'pad_token': PAD_TOKEN, 'mask_token': MASK_TOKEN}
num_added_toks = tokenizer.add_special_tokens(special_tokens_dict)
VOCAB_SIZE = len(tokenizer)
print(VOCAB_SIZE)

52005


##Get a dataloader

In [ ]:
#@title StoryDataLoader
class StoryDataLoader():
 
  def __init__(self, df_filename, tokenizer, batch_size, df_file_len, num_seqs_per_story, context_len, load_stories_at_a_time):
    self.df_filename = df_filename
    self.tokenizer = tokenizer
    self.batch_size = batch_size
    self.total_df_len = df_file_len
    self.remaining_df_items = df_file_len
    self.num_seqs_per_story = num_seqs_per_story
    self.context_len = context_len
    self.all_subsequences_from_story = []
    self.load_stories_at_a_time = load_stories_at_a_time
    self.DF_BUFFER = 100 + self.load_stories_at_a_time
    self.df = self.init_df()
    self.story_buffer = []


  def init_df(self):
     self.remaining_df_items = self.total_df_len
     df = pd.read_csv(self.df_filename, 
                 index_col=0,
                 compression={'method': 'gzip', 'compresslevel': 2}, 
                 chunksize=self.load_stories_at_a_time, 
                 iterator=True)
     return df

  def _get_random_subsequence(self, seq, subsequence_len):
    start_index = random.randint(0, len(seq) - subsequence_len)
    return seq[start_index:start_index + subsequence_len]

  def has_next_batch(self):
    return self.remaining_df_items > self.DF_BUFFER

  def get_next_story(self):
    if len(self.story_buffer) <= 0:
      chunk = self.df.get_chunk()
      for item in chunk['full_text']:
        self.story_buffer.append(str(item))
    return self.story_buffer.pop()
  
  def get_tokens(self):
    # tokens = []
    while True:
      try:
        story = self.df.get_next_story()
        self.remaining_df_items -= 1 #self.load_stories_at_a_time
      except:
        self.remaining_df_items = 0
        raise Exception("Next batch")
          
      text = story
      tokens = tokenizer.encode(text)
      # tokens = torch.Tensor(tokens).to(torch.long).to(device)
      tokens = [tokenizer.bos_token_id] + tokens + [tokenizer.eos_token_id] + self.context_len * [tokenizer.pad_token_id]
      # print(len(tokens))
      if len(tokens) <= self.context_len:
        print("Story too short, retrying now")
        continue
      else:
        return tokens

  def get_next_batch(self):
    batch = []
    while len(batch) < self.batch_size:
      if len(self.all_subsequences_from_story) == 0:
        tokens = self.get_tokens()
        # tokens = tokens + [tokenizer.pad_token_id] * self.context_len
        # print(tokens)

        for _ in range(self.num_seqs_per_story):
          subseq = self._get_random_subsequence(tokens, self.context_len-1)
          subseq = [tokenizer.cls_token_id] + subseq
          self.all_subsequences_from_story.append(subseq)
      subseq = self.all_subsequences_from_story.pop()
      batch.append(subseq)
    if len(batch) > self.batch_size:
      batch = batch[0:self.batch_size]
    return batch


In [ ]:
#@title SingleStoryDataLoader
class SingleStoryDataLoader():
 
  def __init__(self, text_filename, tokenizer, batch_size, context_len):
    self.text_filename = text_filename
    self.tokenizer = tokenizer
    self.batch_size = batch_size
    self.context_len = context_len
    self.all_text_tokens = self.get_text_tokens()


  def get_text_tokens(self):
    with open(self.text_filename) as text_file:
      uncleaned_text = text_file.read()
    tokens = tokenizer.encode(uncleaned_text)
    tokens = [tokenizer.bos_token_id] + tokens + [tokenizer.eos_token_id] + self.context_len * [tokenizer.pad_token_id]
    return tokens


  def _get_random_subsequence(self, seq, subsequence_len):
    start_index = random.randint(0, len(seq) - subsequence_len)
    return seq[start_index:start_index + subsequence_len]

  def has_next_batch(self):
    return True

  def get_next_batch(self):
    batch = []
    while len(batch) < self.batch_size:
      subseq = self._get_random_subsequence(self.all_text_tokens, self.context_len-1)
      subseq = [tokenizer.cls_token_id] + subseq
      batch.append(subseq)
    if len(batch) > self.batch_size:
      batch = batch[0:self.batch_size]
    return batch


In [ ]:
#@title SingleStoryBertAssistDataLoader
class SingleStoryBertAssistDataLoader():
 
  def __init__(self, text_filename, tokenizer, batch_size, context_len, bert_model, mask_percentage=.15):
    self.text_filename = text_filename
    self.tokenizer = tokenizer
    self.batch_size = batch_size
    self.context_len = context_len
    self.all_text_tokens = self.get_text_tokens()
    self.bert_model = bert_model
    self.mask_percentage = mask_percentage
    self.mask_token_id = self.tokenizer.mask_token_id # bert_model.tokenizer.mask_token_id

  def get_text_tokens(self):
    with open(self.text_filename) as text_file:
      uncleaned_text = text_file.read()
    tokens = self.tokenizer.encode(uncleaned_text)
    tokens = tokens + self.context_len * [self.tokenizer.pad_token_id]
    return tokens

  def _get_random_subsequence(self, seq, subsequence_len):
    start_index = random.randint(0, len(seq) - subsequence_len)
    return seq[start_index:start_index + subsequence_len]

  def _augment_subsequence(self, seq):
    rand = torch.rand(len(seq))
    mask_selection = rand < self.mask_percentage
    mask_selection = torch.flatten(mask_selection.nonzero()).tolist()
    # print(mask_selection)
    # mask_selection = [2,4,6,8]
    # old_ids = []
    # print("Pre-change text: ",self.tokenizer.decode(seq) )
    for idx in mask_selection:
      # old_ids.append(seq[idx])
      seq[idx] = self.mask_token_id
    # text = self.tokenizer.decode(seq)
    processed_seq = torch.Tensor([seq]).to(torch.long).to(device)
    # print(seq)
    # print("Conversion text: ",self.tokenizer.decode(seq) )
    new_words = self.bert_model(processed_seq)
    # print(unmasked)
    new_words = new_words['logits'][0]
    # print(new_words)
    # print(seq)
    for idx, new_word in enumerate(new_words):
      if idx not in mask_selection:
        continue
      else:
        new_word_id = torch.argmax(new_word)
        # print(seq, new_word_id)
        seq[idx] = int(new_word_id)
    return seq

  def has_next_batch(self):
    return True

  def get_next_batch(self):
    batch = []
    while len(batch) < self.batch_size:
      subseq = self._get_random_subsequence(self.all_text_tokens, self.context_len-1)
      subseq = self._augment_subsequence(subseq)
      subseq = [tokenizer.cls_token_id] + subseq + 50 * [tokenizer.pad_token_id]
      subseq = subseq[0:self.context_len]
      batch.append(subseq)
    if len(batch) > self.batch_size:
      batch = batch[0:self.batch_size]
    return batch

#Set up the model

In [7]:
test_parts = True
device= "cuda" if torch.cuda.is_available() else "cpu"

In [15]:
run_number = 3
previous_run_numbers = [0,1,2]
if run_number in previous_run_numbers:
  raise(Exception("Pick a new run number!"))

run_name = f"bert_init_test_{run_number}"

wandb.init(project="bert_experiments", 
           name=run_name,
           )

wandb.config = {
  "epochs": 5,
  "batch_size": None,
  "val_batch_size": None,
  "seed":135,
  # Model info
  "d_model":512,
  "vocab_length": VOCAB_SIZE,
  "context_len": 256,
  "num_layers":12,
  "feed_forward_dimensions":1024,
  "attention_heads":12,
  "attention_qkv_dims":64,
  "dropout":.1,
  "device":device,
  #Training config
  "lr": 1e-4,
  # "stories_per_epoch": 5000,
  "save_model": None,
  "bert_mask_percentage": None,
}

config = wandb.config

random.seed(config['seed'])
torch.random.manual_seed(config['seed'])

print(config)

{'epochs': 5, 'batch_size': None, 'val_batch_size': None, 'seed': 135, 'd_model': 512, 'vocab_length': 52005, 'context_len': 256, 'num_layers': 12, 'feed_forward_dimensions': 1024, 'attention_heads': 12, 'attention_qkv_dims': 64, 'dropout': 0.1, 'device': 'cpu', 'lr': 0.0001, 'save_model': None, 'bert_mask_percentage': None}


In [16]:
model = bert.BertModel(d_model = config["d_model"], 
                        vocab_length = config["vocab_length"], 
                        sequence_length = config['context_len'],
                        num_layers = config["num_layers"], 
                        feed_forward_dimensions = config["feed_forward_dimensions"], 
                        attention_heads = config["attention_heads"],
                        attention_qkv_dims =  config["attention_qkv_dims"], 
                        dropout = config["dropout"], 
                        pad_idx = tokenizer.pad_token_id, 
                        device = config["device"])

In [17]:
if test_parts:
    inp = torch.LongTensor([[0,1,2,3,4,5,6,7,8,29],[1,3,4,5,6,8,0,22,1,1]])
    out = model(inp)
    print(out.size())

torch.Size([2, 10, 52005])


In [18]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [19]:
batch = torch.LongTensor([[0,1,2,3,4,5,6,7,8,29],[1,3,4,5,6,8,0,22,1,1]])
print(batch.size())
# input_ids, segment_ids, masked_tokens, masked_pos, isNext = map(torch.LongTensor, zip(*batch))

for epoch in range(config["epochs"]):
    optimizer.zero_grad()
    logits_lm = model(batch)
    print(logits_lm.size(), batch.size())
    loss_lm = criterion(logits_lm.transpose(1, 2), batch) # for masked LM
    loss_lm = (loss_lm.float()).mean()
    # loss_clsf = criterion(logits_clsf, isNext) # for sentence classification
    loss = loss_lm #+ loss_clsf
    if (epoch + 1) % 10 == 0:
        print('Epoch:', '%04d' % (epoch + 1), 'loss =', '{:.6f}'.format(loss))
    loss.backward()
    optimizer.step()

# # Predict mask tokens
# input_ids, segment_ids, masked_tokens, masked_pos, isNext = map(torch.LongTensor, zip(batch[0]))
# print(text)
# print([number_dict[w.item()] for w in input_ids[0] if number_dict[w.item()] != '[PAD]'])

# logits_lm, logits_clsf = model(input_ids, segment_ids, masked_pos)
# logits_lm = logits_lm.data.max(2)[1][0].data.numpy()
# print('masked tokens list : ',[pos.item() for pos in masked_tokens[0] if pos.item() != 0])
# print('predict masked tokens list : ',[pos for pos in logits_lm if pos != 0])

# logits_clsf = logits_clsf.data.max(1)[1].data.numpy()[0]
# print('isNext : ', True if isNext else False)
# print('predict isNext : ',True if logits_clsf else False)

torch.Size([2, 10])
torch.Size([2, 10, 52005]) torch.Size([2, 10])
torch.Size([2, 10, 52005]) torch.Size([2, 10])
torch.Size([2, 10, 52005]) torch.Size([2, 10])
torch.Size([2, 10, 52005]) torch.Size([2, 10])
torch.Size([2, 10, 52005]) torch.Size([2, 10])
